In [1]:
from model.MF import *
from preprocess.Yelp import *
from evaluation.MF_evaluation import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/yelp2018'
dataset = Yelp(path)

# Data(num_nodes=144242, edge_index=[2, 2380730], edge_label_index=[2, 603378])
data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'learning_rate': 1e-5,  # over-fitting
    'epochs': 100,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
    "weight_decay": 1e-7,
    'global_bias':(data.edge_index.size(1) + data.edge_label_index.size(1) + 2) / (num_books * num_users)
}
model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

In [2]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

In [3]:
config['epochs'] = 500
retrain_model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)
retrain_model, recall, ndcg = MF_based_eva(retrain_model, config, retain_data, device)

Epoch 1/500, Train Loss: 0.9961, HR@20: 0.0003, Recall@20: 0.0005, NDCG@20: 0.0004
Epoch 2/500, Train Loss: 0.9932, HR@20: 0.0003, Recall@20: 0.0005, NDCG@20: 0.0004
Epoch 3/500, Train Loss: 0.9903, HR@20: 0.0003, Recall@20: 0.0005, NDCG@20: 0.0004
Epoch 4/500, Train Loss: 0.9875, HR@20: 0.0003, Recall@20: 0.0006, NDCG@20: 0.0005
Epoch 5/500, Train Loss: 0.9846, HR@20: 0.0003, Recall@20: 0.0006, NDCG@20: 0.0005
Epoch 6/500, Train Loss: 0.9818, HR@20: 0.0003, Recall@20: 0.0006, NDCG@20: 0.0005
Epoch 7/500, Train Loss: 0.9789, HR@20: 0.0003, Recall@20: 0.0006, NDCG@20: 0.0005
Epoch 8/500, Train Loss: 0.9761, HR@20: 0.0003, Recall@20: 0.0007, NDCG@20: 0.0005
Epoch 9/500, Train Loss: 0.9732, HR@20: 0.0003, Recall@20: 0.0007, NDCG@20: 0.0005
Epoch 10/500, Train Loss: 0.9704, HR@20: 0.0003, Recall@20: 0.0007, NDCG@20: 0.0005
Epoch 11/500, Train Loss: 0.9676, HR@20: 0.0003, Recall@20: 0.0007, NDCG@20: 0.0005
Epoch 12/500, Train Loss: 0.9648, HR@20: 0.0004, Recall@20: 0.0007, NDCG@20: 0.0006
E

In [4]:
MF_forget_data_eva(retrain_model, None, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0008, Recall@20: 0.0036, NDCG@20: 0.0018


In [6]:
config['epochs'] = 100
# Define the model
teacher = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)
student = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

# Load the model
teacher.load_state_dict(torch.load(f"MF_Yelp2018_{config['epochs']}_Epochs_Top_{config['k']}.pt"))
student.load_state_dict(torch.load(f"MF_Yelp2018_{config['epochs']}_Epochs_Top_{config['k']}.pt"))

<All keys matched successfully>

In [8]:
# Setting the basic hyperparameters
config['beta'] = 0.7
config['alpha'] = 0.3
config['epochs'] = 50
config['gamma'] = 1e-6 # contrastive loss
config['delta'] = 1e-3 # regularization loss
config['tuning_type'] = 'gpf'
config['weight_decay'] = 0.001
config['regularization'] = False
config['Contrastive_loss'] = False
student, prompt= prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

c:\Users\MSI\.conda\envs\master\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_label_index', 'x', 'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Epoch 1/50, Train Loss: 0.6953, HR@20: 0.0109, Recall@20: 0.0214, NDCG@20: 0.0183
Epoch 2/50, Train Loss: 0.6953, HR@20: 0.0109, Recall@20: 0.0214, NDCG@20: 0.0183
Epoch 3/50, Train Loss: 0.6953, HR@20: 0.0109, Recall@20: 0.0214, NDCG@20: 0.0183
Epoch 4/50, Train Loss: 0.6953, HR@20: 0.0109, Recall@20: 0.0214, NDCG@20: 0.0183
Epoch 5/50, Train Loss: 0.6952, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 6/50, Train Loss: 0.6952, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 7/50, Train Loss: 0.6952, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 8/50, Train Loss: 0.6952, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 9/50, Train Loss: 0.6952, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 10/50, Train Loss: 0.6952, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 11/50, Train Loss: 0.6952, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 12/50, Train Loss: 0.6951, HR@20: 0.0109, Recall@20: 0.0213, NDCG@20: 0.0182
Epoch 13/50, 

In [9]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0059, Recall@20: 0.0287, NDCG@20: 0.0163


In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = False
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = False

student, prompt= prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

Epoch 1/50, Train Loss: 0.6969, HR@20: 0.0105, Recall@20: 0.0210, NDCG@20: 0.0173
Epoch 2/50, Train Loss: 0.6969, HR@20: 0.0105, Recall@20: 0.0210, NDCG@20: 0.0172
Epoch 3/50, Train Loss: 0.6969, HR@20: 0.0105, Recall@20: 0.0209, NDCG@20: 0.0172
Epoch 4/50, Train Loss: 0.6969, HR@20: 0.0105, Recall@20: 0.0209, NDCG@20: 0.0172
Epoch 5/50, Train Loss: 0.6969, HR@20: 0.0105, Recall@20: 0.0209, NDCG@20: 0.0172
Epoch 6/50, Train Loss: 0.6969, HR@20: 0.0105, Recall@20: 0.0209, NDCG@20: 0.0172
Epoch 7/50, Train Loss: 0.6969, HR@20: 0.0105, Recall@20: 0.0209, NDCG@20: 0.0172
Epoch 8/50, Train Loss: 0.6969, HR@20: 0.0104, Recall@20: 0.0208, NDCG@20: 0.0171
Epoch 9/50, Train Loss: 0.6969, HR@20: 0.0104, Recall@20: 0.0208, NDCG@20: 0.0171
Epoch 10/50, Train Loss: 0.6968, HR@20: 0.0104, Recall@20: 0.0208, NDCG@20: 0.0171
Epoch 11/50, Train Loss: 0.6968, HR@20: 0.0104, Recall@20: 0.0207, NDCG@20: 0.0171
Epoch 12/50, Train Loss: 0.6968, HR@20: 0.0104, Recall@20: 0.0207, NDCG@20: 0.0171
Epoch 13/50, 

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

In [ ]:
config['tuning_type'] = 'gpf-plus'
config["number_p"] = 10 # The number of prompts
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)